In [1]:
import requests

requestURL = "https://www.ebi.ac.uk/proteins/api/features?accession=P04637"

r = requests.get(requestURL, headers={ "Accept" : "application/json"})

if not r.ok:
    r.raise_for_status()

responseBody = r.text
print(responseBody)


[{"accession":"P04637","entryName":"P53_HUMAN","sequence":"MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD","sequenceChecksum":"AD5C149FD8106131","taxid":9606,"features":[{"type":"CHAIN","category":"MOLECULE_PROCESSING","ftId":"PRO_0000185703","description":"Cellular tumor antigen p53","begin":"1","end":"393","molecule":""},{"type":"DNA_BIND","category":"DOMAINS_AND_SITES","description":"","begin":"102","end":"292","molecule":"","evidences":[{"code":"ECO:0000269","source":{"name":"PubMed","id":"16793544","url":"http://www.ncbi.nlm.nih.gov/pubmed/16793544","alternativeUrl":"https://europepmc.org/abstract/MED/16793544"}},{"code":"ECO:0000269","sour

In [12]:
#задание 2
import requests

def get_protein_length(accession):
    requestURL = f"https://www.ebi.ac.uk/proteins/api/features?accession={accession}"
    r = requests.get(requestURL, headers={"Accept": "application/json"})
    
    if not r.ok:
        r.raise_for_status()

    data = r.json()
  
    if data and 'sequence' in data[0]:
        sequence = data[0]['sequence']
        if sequence:
            length = len(sequence)
            return length
    
    print(f"Длина не найдена для {accession}")
    return None

length1 = get_protein_length("P05067")
length2 = get_protein_length("P10636")

print(f"Длина белка APP (P05067): {length1} аминокислот")
print(f"Длина белка Tau (P10636): {length2} аминокислот")

if length1 and length2:
    if length1 > length2:
        difference = length1 - length2
        print(f"Белок APP длиннее на {difference} аминокислот")
    elif length2 > length1:
        difference = length2 - length1
        print(f"Белок Tau длиннее на {difference} аминокислот")
    else:
        print("Белки имеют одинаковую длину")
else:
    print('Длину белка вычислить нельзя')

Длина белка APP (P05067): 770 аминокислот
Длина белка Tau (P10636): 758 аминокислот
Белок APP длиннее на 12 аминокислот


In [18]:
#задание 3
#функция для получения информации о белке по его accession IDs
#в качестве базы данных используем UniProt 

import requests
import pandas as pd

proteins = ['P08647', 'Q7TS55', 'P20607', 'P35968', 'Q29537']

data = []

for protein_id in proteins:
    url = f'https://rest.uniprot.org/uniprotkb/{protein_id}'
    response=requests.get(url, headers={'Accept': 'application/json'}) 

    if response.ok:
        protein_data=response.json()
        data.append({ 'accesion': protein_id, 'entryName': protein_data.get('uniProtkbId'),
        'length': protein_data.get('sequence', {}).get('length') })

df = pd.DataFrame(data)
print(df)
df.to_csv('protein_data.csv', index=False)

  accesion    entryName  length
0   P08647    RAS_SCHPO     219
1   Q7TS55  TNF18_MOUSE     173
2   P20607      IL6_RAT     211
3   P35968  VGFR2_HUMAN    1356
4   Q29537    P53_CANLF     381


In [21]:
#задание 4

import requests

#поиск ID статей
search_response = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi", 
                               params={'db': 'pubmed', 'term': 'COVID-19 variants',
                                       'sort': 'date', 'retmax': 10, 'retmode': 'json'})
 
article_ids = search_response.json()['esearchresult']['idlist']

details_response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",
    params={'db': 'pubmed', 'id': ','.join(article_ids), 'retmode': 'json'})

articles = details_response.json()['result']

print("10 последних статей по запросу 'COVID-19 variants':\n")
for i, article_id in enumerate(article_ids, 1):
    title = articles[article_id].get('title', 'заголовок недоступен')
    print(f"{i}. PubMed ID: {article_id}")
    print(f"{title}\n")


10 последних статей по запросу 'COVID-19 variants':

1. PubMed ID: 41175012
SARS-CoV-2 Evolution in Humans Enables its Transmission to Nonhuman Primates.

2. PubMed ID: 41174545
Comparison of fully-vaccinated and non/partially-vaccinated critically-ill COVID-19 patients in terms of disease severity: a retrospective observational study.

3. PubMed ID: 41174076
Oral health, smoking status, and oral manifestations of adult patients with SARS-CoV-2 during the Omicron outbreak in China: a cross-sectional study.

4. PubMed ID: 41171875
One SQ HEDGES DNA vector only dose produces durable hGLA or anti-SARS-CoV-2 mAb therapeutic serum protein levels.

5. PubMed ID: 41170974
Tissue tropism and functional adaptation of the SARS-CoV-2 spike protein in a fatal case of COVID-19.

6. PubMed ID: 41170164
Genetic Risk of Different Populations in COVID-19 Susceptibility and Severity.

7. PubMed ID: 41168466
Uncovering potent natural phytochemicals targeting SARS-COV-2 spike protein variants: molecular d

In [29]:
#задание 5

import requests

def get_protein_name(accession):
    url = f"https://rest.uniprot.org/uniprotkb/{accession}"
    response = requests.get(url, headers={'Accept': 'application/json'})
    
    if response.status_code == 200:
        protein_data = response.json()
        entry_name = protein_data.get('uniProtkbId')
        return entry_name if entry_name else "Название не найдено"
    elif response.status_code == 404:
        return "Белок не найден"
    else:
        return "Ошибка при запросе"

print(get_protein_name('P20607'))

IL6_RAT


In [33]:
#задание 6

import requests

with open('accessions.txt', 'w') as f:
    f.write('P08647\nQ7TS55\nP20607\nP35968\nQ29537')
with open('accessions.txt', 'r') as f:
    accessions = [line.strip() for line in f if line.strip()]

with open('proteins_info.txt', 'w') as output_file:
    output_file.write('accessions\tentryName\torganism\n')
    for accession in accessions:
        url = f"https://rest.uniprot.org/uniprotkb/{accession}"
        response = requests.get(url, headers = {'accept': 'application/json'})
        if response.status_code == 200:
            data = response.json()
            entry_name = data.get('uniProtkbId', 'N/A')
            organism = data.get('organism', {}).get('scientificName', 'N/A')
            output_file.write(f"{accession}\t{entry_name}\t{organism}\n")
            
            print(f"{accession}: {entry_name} - {organism}")
        else:
            output_file.write(f"{accession}\tNot found\tNot found\n")
            print(f"{accession}: не найден")

P08647: RAS_SCHPO - Schizosaccharomyces pombe (strain 972 / ATCC 24843)
Q7TS55: TNF18_MOUSE - Mus musculus
P20607: IL6_RAT - Rattus norvegicus
P35968: VGFR2_HUMAN - Homo sapiens
Q29537: P53_CANLF - Canis lupus familiaris


In [34]:
#задание 7

import requests

ids = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={'db': 'pubmed', 'term': 'COVID-19 variants', 'retmax': 4, 'retmode': 'json'}
).json()['esearchresult']['idlist']

for id in ids:
    xml = requests.get(
        f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
        params={'db': 'pubmed', 'id': id, 'retmode': 'xml'}
    ).text
    
    authors = [line.split('>')[1].split('<')[0] for line in xml.split('\n') if '<LastName>' in line]
    print(f"PMID: {id}, Авторы: {', '.join(authors) if authors else 'Нет данных'}")

PMID: 41175012, Авторы: 
PMID: 41174545, Авторы: 
PMID: 41174076, Авторы: 
PMID: 41171875, Авторы: 


In [37]:
#задание 8

import requests

ids = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={'db': 'pubmed', 'term': 'COVID-19 variants', 'retmax': 4, 'retmode': 'json'}
).json()['esearchresult']['idlist']

print("4 последние статьи по COVID-19 variants:\n")

for article_id in ids:
    summary = requests.get(
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",
        params={'db': 'pubmed', 'id': article_id, 'retmode': 'json'}
    ).json()
    article_info = summary['result'][article_id]
    
    print(f"PMID: {article_id}")
    print(f"Заголовок: {article_info.get('title', 'Нет заголовка')}")
    
    authors = []
    if 'authors' in article_info:
        for author in article_info['authors']:
            authors.append(author.get('name', ''))
 
    print(f"Авторы: {', '.join(authors) if authors else 'Не указаны'}")
    print()

4 последние статьи по COVID-19 variants:

PMID: 41175012
Заголовок: SARS-CoV-2 Evolution in Humans Enables its Transmission to Nonhuman Primates.
Авторы: Chiu YT, Huang YS, Pan MY, Chang CY, Chiu CC, Nguyen V, Lee TH, Sun YJ, Li WH, Wang LH

PMID: 41174545
Заголовок: Comparison of fully-vaccinated and non/partially-vaccinated critically-ill COVID-19 patients in terms of disease severity: a retrospective observational study.
Авторы: Louis G, Dinot V, Belveyre T, Dirand O, Dunand P, Cadoz C, Gaci R, Gette S, Perez P, Goetz C, Picard Y, Mellati N

PMID: 41174076
Заголовок: Oral health, smoking status, and oral manifestations of adult patients with SARS-CoV-2 during the Omicron outbreak in China: a cross-sectional study.
Авторы: Zhang H, Zhang Z, Liu L, Wang X, Xue R, Wang Y, Fan L, Tian X, Zhang Y, Liu R

PMID: 41171875
Заголовок: One SQ HEDGES DNA vector only dose produces durable hGLA or anti-SARS-CoV-2 mAb therapeutic serum protein levels.
Авторы: Ye A, Mack M, Ice R, Heath T, Ericksso

In [58]:
#задание 9 

import requests

accession = 'Q8WZ42'
url = f'https://rest.uniprot.org/uniprotkb/{accession}'

response = requests.get(url, headers={'Accept': 'application/json'})
data = response.json()
sequence = data.get('sequence', {}).get('value')

print('последовательность белка Titin:')
for i in range(0, len(sequence), 80):
    print(sequence[i: i+80])

последовательность белка Titin:
...
QKQDGGLYTLSLGNEFGSDSATVNIHIRSI


In [11]:
#задание 11

import requests
import pandas as pd

gene_name = 'APOE'
#поиск в UniProt
response = requests.get(url = 'https://rest.uniprot.org/uniprotkb/search', 
                        params = {'query': f'gene:{gene_name} AND organism_id:9606',
'format': 'json'})
proteins = []
for item in response.json()['results']:
    protein_desc = item.get('proteinDescription', {}) #если в описании белка ничего нет, возвращаем пустой словарь 
    names = []
    # названия хранятся в 'recommendedName' и 'alternativeNames' - их нужно объединить 
    if 'recommendedName' in protein_desc:
        names.append(protein_desc['recommendedName']['fullName']['value'])
    if 'alternativeNames' in protein_desc:
        for alt_name in protein_desc['alternativeNames']:
            names.append(alt_name['fullName']['value'])
    all_names = '; '.join(names) if names else 'N/A' #если список пустой, возвращаем NA
    
    proteins.append({
        'gene': gene_name,
        'accession': item['primaryAccession'],
        'entry_name': item['uniProtkbId'],
        'Protein Names': all_names,
        'UniProt URL': f"https://www.uniprot.org/uniprot/{item['primaryAccession']}"
    })

pd.DataFrame(proteins).to_csv(f'{gene_name}_proteins.csv', index=False)
print(f"Сохранено {len(proteins)} записей в {gene_name}_proteins.csv")


Сохранено 12 записей в APOE_proteins.csv


In [20]:
#задание 12

import requests
import urllib.parse

# Набор из 100 реальных id белков 
protein_ids = [
    "P05067", "P35579", "P12081", "P16070", "P08100", "P16435", "P02768", "P02647", 
    "P68871", "P69905", "P61769", "P0CG47", "P62979", "P62258", "P61981", "P62333", 
    "P62857", "P62854", "P61201", "P46781", "P46782", "P46783", "P46776", "P46777", 
    "P46778", "P46779", "P46780", "P27361", "P28482", "Q05397", "P12931", "P07900", 
    "P11388", "P04637", "Q07817", "Q07812", "Q07820", "O00255", "O00264", "O14757", 
    "O14763", "O14965", "O14976", "O15287", "O15350", "O15524", "O15530", "O43187", 
    "O43283", "O43318", "O43474", "O43524", "O43795", "O43823", "O43934", "O60260", 
    "O60341", "O60674", "O60701", "O60760", "O60885", "O60936", "O75122", "O75369", 
    "O75400", "O75460", "O75533", "O75676", "O75791", "O75822", "O75923", "O76070", 
    "O76076", "O95292", "O95342", "O95400", "O95402", "O95503", "O95613", "O95631", 
    "P0DTC1", "P0DTD1", "P59594", "P0DTC2", "P0DTC3", "P0DTC4", "P0DTC5", "P0DTC6", 
    "P0DTC7", "P0DTC8", "P0DTC9", "P0DTD2", "P0DTD3", "P0DTD4", "P0DTD5", "P0DTD6", 
    "P0DTD7", "P0DTD9", "P0DTE1", "P0DTE2"
]

accession = ','.join(protein_ids[:100])  
url = f"https://www.ebi.ac.uk/proteins/api/proteins?accession={accession}"

response = requests.get(url, headers={"Accept": "application/json"})

if response.status_code == 200:
    data = response.json()
    print(f"Успешно получены данные по {len(data)} белкам:")
    for protein in data:
        accession = protein.get("accession")
        protein_name = protein.get('protein', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A')
        print(f"{accession}: {protein_name}")
else:
    print(f"Ошибка: {response.status_code}")

Успешно получены данные по 100 белкам:
O00255: Menin
O00264: Membrane-associated progesterone receptor component 1
O14757: Serine/threonine-protein kinase Chk1
O14763: Tumor necrosis factor receptor superfamily member 10B
O14965: Aurora kinase A
O14976: Cyclin-G-associated kinase
O15287: Fanconi anemia group G protein
O15350: Tumor protein p73
O15524: Suppressor of cytokine signaling 1
O15530: 3-phosphoinositide-dependent protein kinase 1
O43187: Interleukin-1 receptor-associated kinase-like 2
O43283: Mitogen-activated protein kinase kinase kinase 13
O43318: Mitogen-activated protein kinase kinase kinase 7
O43474: Krueppel-like factor 4
O43524: Forkhead box protein O3
O43795: Unconventional myosin-Ib
O43823: A-kinase anchor protein 8
O43934: UNC93-like protein MFSD11
O60260: E3 ubiquitin-protein ligase parkin
O60341: Lysine-specific histone demethylase 1A
O60674: Tyrosine-protein kinase JAK2
O60701: UDP-glucose 6-dehydrogenase
O60760: Hematopoietic prostaglandin D synthase
O60885: Brom

In [24]:
#задание 13

import requests
id='P05067'
url = f"https://www.ebi.ac.uk/proteins/api/proteins?accession={id}"

params = {'format': 'fasta'}
response = requests.get(url, params=params)

if response.status_code == 200:
    filename = f'{id}.fasta'
    with open(filename, 'w') as file:
        file.write(response.text)
    print(f'данные о белке сохранены в файл {filename}')
else:
    print(f'ошибка')
    print(response.text)
    

данные о белке сохранены в файл P05067.fasta


In [9]:
#задание 14 

import requests

try:
    #специально попытаемся получить несуществующую страницу
    response = requests.get("https://httpbin.org/status/404")
    response.raise_for_status()  
    
except requests.exceptions.HTTPError:
    print("ошибка 404: страница не найдена")
    print("проверьте правильность адреса")
    
except Exception as e:
    print(f"Произошла ошибка: {e}")

ошибка 404: страница не найдена
проверьте правильность адреса


In [32]:
#задание 15
import requests

gene_name = input('название гена:')

url = "https://rest.uniprot.org/uniprotkb/search"
params = {'query': f'gene:{gene_name}', 'organism': 'human', 'format': 'json', 'size': 1} 
response = requests.get(url, params=params)
data = response.json()

if data['results']:
    protein_id = data['results'][0]['primaryAccession']
    print(f'гену соответствует белок: {protein_id}')
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {'db': 'pubmed', 'term': f'{gene_name}[Gene Name] AND human[Organism]', 
    'retmode': 'json', 'retmax': 5}
    response = requests.get(base_url, params=params)
    data = response.json()
    id_list = data['esearchresult']['idlist']
    print(f'id статей:{id_list}')
else:
    print('белок не найден')


название гена: APOE


гену соответствует белок: A0A0D9S1R0
id статей:['41183695', '41183252', '41181127', '41180196', '41176538']


In [7]:
#задание 16

%%bash

#создадим json файл 
cat > protein_data.json << EOF
{"accession": "P12345", "name": "Example Protein", "organism": "Homo sapiens"}
EOF

json_file="protein_data.json"

# Извлекаем accession ID
if command -v jq &> /dev/null; then
    jq -r '.accession' "$json_file"
else
    python3 -c "import json; print(json.load(open('$json_file')).get('accession', 'Not found'))"
fi

SyntaxError: invalid syntax (2950264887.py, line 14)

In [4]:
#задание 16
%%bash
echo "Hello, world!"

SyntaxError: invalid syntax (556847310.py, line 3)

In [61]:
#задание 17

import requests 

#последние статьи о мутациях p53 в онкологии
url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "pubmed",
    "term": "p53 mutations cancer",
    "retmode": "json",
    "retmax": 3
}

response = requests.get(url, params=params).json()
ids = response["esearchresult"]["idlist"]
print("последние статьи о мутациях p53:")
for id in ids:
    print(f"https://pubmed.ncbi.nlm.nih.gov/{id}/")

#получение общих данных о p53
url = "https://www.ebi.ac.uk/proteins/api/proteins/P04637"
headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers).json()
print(f"ген: {response['gene'][0]['name']['value']}")
print(f"организм: {response['organism']['names'][0]['value']}")

#статьи про клинические исследования с p53
url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "pubmed",
    "term": "p53 clinical trial",
    "retmode": "json",
    "retmax": 3
}

response = requests.get(url, params=params).json()
ids = response["esearchresult"]["idlist"]
print("\nклинические исследования p53:")
for id in ids:
    print(f"https://pubmed.ncbi.nlm.nih.gov/{id}/")

последние статьи о мутациях p53:
https://pubmed.ncbi.nlm.nih.gov/41193733/
https://pubmed.ncbi.nlm.nih.gov/41193463/
https://pubmed.ncbi.nlm.nih.gov/41191814/
ген: TP53
организм: Homo sapiens

клинические исследования p53:
https://pubmed.ncbi.nlm.nih.gov/41171586/
https://pubmed.ncbi.nlm.nih.gov/41171398/
https://pubmed.ncbi.nlm.nih.gov/41118034/
